In [9]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML

# For model building
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Miscelaneous
from IPython.display import display, clear_output
import pdb
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')

X_train = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_train.pkl')
X_validation = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_validation.pkl')
X_augmented_01 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_augmented_01.pkl')
X_augmented_02 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_augmented_02.pkl')
X_augmented_03 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_augmented_03.pkl')
X_augmented_04 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_augmented_04.pkl')
X_validation_augmented_03 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_validation_augmented_03.pkl')
X_test = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_test.pkl')
X_test_augmented_03 = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/X_test_augmented_03.pkl')

In [10]:
# Making single correction to label
correction_idx = X_test.loc[(X_test['Patient']=='14') & (X_test['Type']=='af') & (X_test['Channel']=='CS3-4') &
                            (X_test['Coupling Interval']=='300')].index[0]

X_test.at[correction_idx, 'Label'] = 1

In [11]:
# Isolate feature matrices, target vectors and information for upsampled dataset
X_train_ = X_train.drop(['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_train = X_train['Label'].astype(int)
info_train = X_train[['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_augmented_01_ = X_augmented_01.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_augmented_01 = X_augmented_01['Label'].astype(int)
info_augmented_01 = X_augmented_01[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_augmented_02_ = X_augmented_02.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_augmented_02 = X_augmented_02['Label'].astype(int)
info_augmented_02 = X_augmented_02[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_augmented_03_ = X_augmented_03.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_augmented_03 = X_augmented_03['Label'].astype(int)
info_augmented_03 = X_augmented_03[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_augmented_04_ = X_augmented_04.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_augmented_04 = X_augmented_04['Label'].astype(int)
info_augmented_04 = X_augmented_04[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_validation_ = X_validation.drop(['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_validation = X_validation['Label'].astype(int)
info_validation = X_validation[['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_validation_augmented_03_ = X_validation_augmented_03.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_validation_augmented_03 = X_validation_augmented_03['Label'].astype(int)
info_validation_augmented_03 = X_validation_augmented_03[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_test_augmented_03_ = X_test_augmented_03.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_test_augmented_03 = X_test_augmented_03['Label'].astype(int)
info_test_augmented_03 = X_test_augmented_03[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

X_test_ = X_test.drop(['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
y_test = X_test['Label'].astype(int)
info_test = X_test[['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

In [12]:
X_combined_augmented_03 = pd.concat([X_augmented_03_, X_validation_augmented_03_, X_test_augmented_03_], ignore_index=True)
y_combined_augmented_03 = pd.concat([y_augmented_03, y_validation_augmented_03, y_test_augmented_03], ignore_index=True)
info_combined_augmented_03 = pd.concat([info_augmented_03, info_validation_augmented_03, info_test_augmented_03], ignore_index=True)

X_combined_ = pd.concat([X_train_, X_validation_, X_test_], ignore_index=True)
y_combined = pd.concat([y_train, y_validation, y_test], ignore_index=True)
info_combined = pd.concat([info_train, info_validation, info_test], ignore_index=True)

## Setup Model and Estimators

In [13]:
clf = LogisticRegression(penalty='l1', C=1, random_state=1, solver='saga', multi_class='multinomial', 
                         class_weight='balanced')

### Extract Feature Matrices

In [14]:
X_aug1 = X_augmented_01_.values
X_aug2 = X_augmented_02_.values
X_aug3 = X_augmented_03_.values
X_aug4 = X_augmented_04_.values
X = X_train_.values
X_val = X_validation_.values
X_val_aug3 = X_validation_augmented_03_.values
X_t = X_test_.values
X_t_aug3 = X_test_augmented_03_.values

X_combined_aug3 = X_combined_augmented_03.values
X_combined = X_combined_.values

y_aug1 = y_augmented_01.values
y_aug2 = y_augmented_02.values
y_aug3 = y_augmented_03.values
y_aug4 = y_augmented_04.values
y = y_train.values
y_val = y_validation.values
y_val_aug3 = y_validation_augmented_03.values
y_t = y_test.values
y_combined_aug3 = y_combined_augmented_03.values
y_combined = y_combined.values

X_info = info_train
X_val_info = info_validation
X_test_info = info_test
X_combined_info = info_combined

## Class Prediction Results

In [17]:
clf.fit(X_combined_aug3, y_combined_aug3)

predictions = clf.predict(X)
predictions_val = clf.predict(X_val)
predictions_t = clf.predict(X_t)
predictions_combined = clf.predict(X_combined)

pp = clf.predict_proba(X)
pp_val = clf.predict_proba(X_val)
pp_t = clf.predict_proba(X_t)
pp_combined = clf.predict_proba(X_combined)

print('Training Data Predictions')
cm = confusion_matrix(y, predictions)
print_cm(cm, ['Green', 'Amber','Red'])
f1 = f1_score(y, predictions, average='weighted')
print('F1 Score: ' + str(f1))

print('\nValidation Data Predictions')
cm = confusion_matrix(y_val, predictions_val)
print_cm(cm, ['Green', 'Amber','Red'])
f1 = f1_score(y_val, predictions_val, average='weighted')
print('F1 Score: ' + str(f1))

print('\nCombined Data Predictions')
cm = confusion_matrix(y_combined, predictions_combined)
print_cm(cm, ['Green', 'Amber','Red'])
f1 = f1_score(y_combined, predictions_combined, average='weighted')
print('F1 Score: ' + str(f1))

print('\nTest Data Predictions')
cm = confusion_matrix(y_t, predictions_t)
print_cm(cm, ['Green', 'Amber','Red'])
f1 = f1_score(y_t, predictions_t, average='weighted')
print('F1 Score: ' + str(f1))

Training Data Predictions
     t/p  Green Amber   Red 
    Green 710.0  98.0   2.0 
    Amber  29.0  98.0  20.0 
      Red   0.0   5.0  23.0 
F1 Score: 0.8558177823001853

Validation Data Predictions
     t/p  Green Amber   Red 
    Green 242.0  23.0   1.0 
    Amber   8.0  26.0   9.0 
      Red   0.0   1.0  12.0 
F1 Score: 0.8772086100495843

Combined Data Predictions
     t/p  Green Amber   Red 
    Green 1798.0 224.0   5.0 
    Amber  60.0 248.0  55.0 
      Red   0.0   8.0  66.0 
F1 Score: 0.8679522029985935

Test Data Predictions
     t/p  Green Amber   Red 
    Green 846.0 103.0   2.0 
    Amber  23.0 124.0  26.0 
      Red   0.0   2.0  31.0 
F1 Score: 0.8756110476322128


In [18]:
X_info['Green PP'] = pp[:,0]
X_info['Amber PP'] = pp[:,1]
X_info['Red PP'] = pp[:,2]
X_info['Predicitions'] = predictions

X_val_info['Green PP'] = pp_val[:,0]
X_val_info['Amber PP'] = pp_val[:,1]
X_val_info['Red PP'] = pp_val[:,2]
X_val_info['Predicitions'] = predictions_val

X_test_info['Green PP'] = pp_t[:,0]
X_test_info['Amber PP'] = pp_t[:,1]
X_test_info['Red PP'] = pp_t[:,2]
X_test_info['Predicitions'] = predictions_t

X_combined_info['Green PP'] = pp_combined[:,0]
X_combined_info['Amber PP'] = pp_combined[:,1]
X_combined_info['Red PP'] = pp_combined[:,2]
X_combined_info['Predicitions'] = predictions_combined

X_info['Weighted PP'] = pp[:,1] + 2*pp[:,2]
X_val_info['Weighted PP'] = pp_val[:,1] + 2*pp_val[:,2]
X_test_info['Weighted PP'] = pp_t[:,1] + 2*pp_t[:,2]
X_combined_info['Weighted PP'] = pp_combined[:,1] + 2*pp_combined[:,2]

## AF vs Non-AF Training Data Predictions

#### Individual Patients

In [13]:
%matplotlib qt
channels = X_combined['Channel'].unique()
patient_types = X_combined['Type'].unique()

for patient_type in patient_types:
    patients = X_combined_info[X_combined_info['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        X_patient = X_combined_info[(X_combined_info['Type']==patient_type) & (X_combined_info['Patient']==patient)]
        
        fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(16,9))
        for channel, ax in zip(channels, axes):
            channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
            x = channel_cis.astype(int)
            y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values
            
            ax.set_title(patient_type + patient + ': ' + channel)
            ax.set_ylabel('Weighted Prediction')
            ax.plot(x, y)
            ax.set_xlim(400, 220)
            ax.set_ylim(0, 2)
            ax.grid(True)
        
        ax.set_xlabel('Coupling Interval')
        plt.draw()
        plt.waitforbuttonpress()
        plt.close()

#### AF vs non-AF

In [21]:
from collections import defaultdict
%matplotlib qt

channels = X_combined_info['Channel'].unique()
patient_types = X_combined_info['Type'].unique()
all_cis = X_combined_info['Coupling Interval'].unique().astype(int)
all_cis = np.sort(all_cis)[::-1]

af_f_values = defaultdict(list)
non_af_f_values = defaultdict(list)

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10,6), dpi=80)

for patient_type in patient_types:
    patients = X_combined_info[X_combined_info['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        X_patient = X_combined_info[(X_combined_info['Type']==patient_type) & (X_combined_info['Patient']==patient)]
        
        if patient_type == 'af':
            for channel, ax in zip(channels, axes[:,0]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values       
                ax.plot(x, y, 'tab:red', alpha=0.3)
                for i, ci in enumerate(channel_cis):
                    af_f_values[(channel + ' ' + str(ci))].append(y[i])
                
        else:
            for channel, ax in zip(channels, axes[:,1]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values
                ax.plot(x, y, 'tab:green', alpha=0.3)
                for i, ci in enumerate(channel_cis):
                    non_af_f_values[(channel + ' ' + str(ci))].append(y[i])

                    

cs12_af_f_values = [af_f_values[('CS1-2' + ' ' + str(ci))] for ci in all_cis]
cs34_af_f_values = [af_f_values[('CS3-4' + ' ' + str(ci))] for ci in all_cis]
cs56_af_f_values = [af_f_values[('CS5-6' + ' ' + str(ci))] for ci in all_cis]

cs12_non_af_f_values = [non_af_f_values[('CS1-2' + ' ' + str(ci))] for ci in all_cis]
cs34_non_af_f_values = [non_af_f_values[('CS3-4' + ' ' + str(ci))] for ci in all_cis]
cs56_non_af_f_values = [non_af_f_values[('CS5-6' + ' ' + str(ci))] for ci in all_cis]
   
cs12_af_f_means = [np.median(l) for l in cs12_af_f_values]
cs34_af_f_means = [np.median(l) for l in cs34_af_f_values]
cs56_af_f_means = [np.median(l) for l in cs56_af_f_values]

cs12_non_af_f_means = [np.median(l) for l in cs12_non_af_f_values]
cs34_non_af_f_means = [np.median(l) for l in cs34_non_af_f_values]
cs56_non_af_f_means = [np.median(l) for l in cs56_non_af_f_values]

axes[0,0].plot(all_cis[cs12_af_f_means != np.nan].reshape(19), cs12_af_f_means, 'k', linewidth=2)
axes[1,0].plot(all_cis[cs34_af_f_means != np.nan].reshape(19), cs34_af_f_means, 'k', linewidth=2)
axes[2,0].plot(all_cis[cs56_af_f_means != np.nan].reshape(19), cs56_af_f_means, 'k', linewidth=2)

axes[0,1].plot(all_cis[cs12_non_af_f_means != np.nan].reshape(19), cs12_non_af_f_means, 'k', linewidth=2)
axes[1,1].plot(all_cis[cs34_non_af_f_means != np.nan].reshape(19), cs34_non_af_f_means, 'k', linewidth=2)
axes[2,1].plot(all_cis[cs56_non_af_f_means != np.nan].reshape(19), cs56_non_af_f_means, 'k', linewidth=2)


axes[0,0].set_title('AF Patients', fontsize=14)
axes[0,1].set_title('Non-AF Patients', fontsize=14)

axes[2,0].set_xlabel('S1/S2 Interval (ms)', fontsize=12)
axes[2,1].set_xlabel('S1/S2 Interval (ms)', fontsize=12)

for channel, ax in zip(channels, axes[:,0]):
    ax.set_ylabel((' $f_{score}$'), fontsize=12)
    
for i, channel in enumerate(channels):
    for ax in axes[i,:]:
        ax.legend([ax.lines[-1]], [(channel + ' Median')])
#     ax.annotate(channel, xy=(50,90), xycoords='axes points',
#             size=12, ha='right', va='top',
#             bbox=dict(boxstyle='round', fc='w'))
    
#     ax.set_ylabel((' $f_{score}$'), fontsize=12)

for ax in axes.ravel():
    ax.spines["top"].set_alpha(0.0)    
    ax.spines["bottom"].set_alpha(0.3)
    ax.spines["right"].set_alpha(0.0)    
    ax.spines["left"].set_alpha(0.3)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    ax.set_axisbelow(True)
#     ax.tick_params(axis='both', which='major', labelsize=12)
  
plt.tight_layout()
plt.show()

#### Same Plot For Popular Feature Values

In [38]:
from collections import defaultdict
%matplotlib qt

feature = 'Number of Peaks'

channels = X_combined_info['Channel'].unique()
patient_types = X_combined_info['Type'].unique()
all_cis = X_combined_info['Coupling Interval'].unique().astype(int)
all_cis = np.sort(all_cis)[::-1]

af_feature_values = defaultdict(list)
non_af_feature_values = defaultdict(list)

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10,6), dpi=80)

for patient_type in patient_types:
    patients = X_combined_info[X_combined_info['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        X_patient = X_combined_info[(X_combined_info['Type']==patient_type) & (X_combined_info['Patient']==patient)]
        X_patient_features = X_combined_[(X_combined_info['Type']==patient_type) & (X_combined_info['Patient']==patient)]
        
        if patient_type == 'af':
            for channel, ax in zip(channels, axes[:,0]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
                x = channel_cis.astype(int)
                y = X_patient_features[X_patient['Channel']==channel][feature].values       
                ax.plot(x, y, 'tab:red', alpha=0.3)
                for i, ci in enumerate(channel_cis):
                    af_feature_values[(channel + ' ' + str(ci))].append(y[i])
                
        else:
            for channel, ax in zip(channels, axes[:,1]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval']
                x = channel_cis.astype(int)
                y = X_patient_features[X_patient['Channel']==channel][feature].values
                ax.plot(x, y, 'tab:green', alpha=0.3)
                for i, ci in enumerate(channel_cis):
                    non_af_feature_values[(channel + ' ' + str(ci))].append(y[i])

                    

cs12_af_feature_values = [af_feature_values[('CS1-2' + ' ' + str(ci))] for ci in all_cis]
cs34_af_feature_values = [af_feature_values[('CS3-4' + ' ' + str(ci))] for ci in all_cis]
cs56_af_feature_values = [af_feature_values[('CS5-6' + ' ' + str(ci))] for ci in all_cis]

cs12_non_af_feature_values = [non_af_feature_values[('CS1-2' + ' ' + str(ci))] for ci in all_cis]
cs34_non_af_feature_values = [non_af_feature_values[('CS3-4' + ' ' + str(ci))] for ci in all_cis]
cs56_non_af_feature_values = [non_af_feature_values[('CS5-6' + ' ' + str(ci))] for ci in all_cis]
   
cs12_af_f_means = [np.median(l) for l in cs12_af_feature_values]
cs34_af_f_means = [np.median(l) for l in cs34_af_feature_values]
cs56_af_f_means = [np.median(l) for l in cs56_af_feature_values]

cs12_non_af_f_means = [np.median(l) for l in cs12_non_af_feature_values]
cs34_non_af_f_means = [np.median(l) for l in cs34_non_af_feature_values]
cs56_non_af_f_means = [np.median(l) for l in cs56_non_af_feature_values]

axes[0,0].plot(all_cis[cs12_af_f_means != np.nan].reshape(19), cs12_af_f_means, 'k', linewidth=2)
axes[1,0].plot(all_cis[cs34_af_f_means != np.nan].reshape(19), cs34_af_f_means, 'k', linewidth=2)
axes[2,0].plot(all_cis[cs56_af_f_means != np.nan].reshape(19), cs56_af_f_means, 'k', linewidth=2)

axes[0,1].plot(all_cis[cs12_non_af_f_means != np.nan].reshape(19), cs12_non_af_f_means, 'k', linewidth=2)
axes[1,1].plot(all_cis[cs34_non_af_f_means != np.nan].reshape(19), cs34_non_af_f_means, 'k', linewidth=2)
axes[2,1].plot(all_cis[cs56_non_af_f_means != np.nan].reshape(19), cs56_non_af_f_means, 'k', linewidth=2)


axes[0,0].set_title('AF Patients', fontsize=14)
axes[0,1].set_title('Non-AF Patients', fontsize=14)

axes[2,0].set_xlabel('S1/S2 Interval (ms)', fontsize=12)
axes[2,1].set_xlabel('S1/S2 Interval (ms)', fontsize=12)

for channel, ax in zip(channels, axes[:,0]):
    ax.set_ylabel('Number of Peaks', fontsize=10)
    
for i, channel in enumerate(channels):
    for ax in axes[i,:]:
        ax.legend([ax.lines[-1]], [(channel + ' Median')], fontsize=12, loc=2)
#     ax.annotate(channel, xy=(50,90), xycoords='axes points',
#             size=12, ha='right', va='top',
#             bbox=dict(boxstyle='round', fc='w'))
    
#     ax.set_ylabel((' $f_{score}$'), fontsize=12)

for ax in axes.ravel():
    ax.spines["top"].set_alpha(0.0)    
    ax.spines["bottom"].set_alpha(0.3)
    ax.spines["right"].set_alpha(0.0)    
    ax.spines["left"].set_alpha(0.3)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 10)
    ax.grid(True)
    ax.set_axisbelow(True)
#     ax.tick_params(axis='both', which='major', labelsize=12)
  
plt.tight_layout()
plt.show()

#### AF vs Non-AF Test Predictions

In [12]:
%matplotlib qt
channels = X_test_info['Channel'].unique()
patient_types = X_test_info['Type'].unique()

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(16,9))

axes[0,0].set_title('AF Patients')
axes[0,1].set_title('Non-AF Patients')

for channel, ax in zip(channels, axes[:,0]):
    ax.set_ylabel(channel)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    
for channel, ax in zip(channels, axes[:,1]):
    ax.set_ylabel(channel)
    ax.set_xlim(400, 220)
    ax.set_ylim(0, 2)
    ax.grid(True)
    
axes[2,0].set_xlabel('Coupling Interval')
axes[2,1].set_xlabel('Coupling Interval')

for patient_type in patient_types:
    patients = X_test_info[X_test_info['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        X_patient = X_test_info[(X_test_info['Type']==patient_type) & (X_test_info['Patient']==patient)]
        
        if patient_type == 'af':
            for channel, ax in zip(channels, axes[:,0]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval'].values
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values       
                ax.plot(x, y, 'C2')
                patient_name = patient_type + patient
                ax.annotate(patient_name, xy=[x[-1],y[-1]])
                
        else:
            for channel, ax in zip(channels, axes[:,1]):
                channel_cis = X_patient[X_patient['Channel']==channel]['Coupling Interval'].values
                x = channel_cis.astype(int)
                y = X_patient[X_patient['Channel']==channel]['Weighted PP'].values
                ax.plot(x, y, 'C3')
                patient_name = patient_type + patient
                ax.annotate(patient_name, xy=[x[-1],y[-1]])

plt.show()

#### Max $f_{score}$ and Conduction Delay

In [137]:
X_all_ = pd.concat([X_combined_, X_test_], ignore_index=True)
X_all_info = pd.concat([X_combined_info, X_test_info], ignore_index=True)
X_all = pd.concat([X_all_, X_all_info], axis=1)

In [ ]:
patients = {}
patients['af'] = np.sort([np.int(x) for x in (X_all[X_all['Type']=='af']['Patient'].unique())])
patients['at'] = np.sort([np.int(x) for x in (X_all[X_all['Type']=='at']['Patient'].unique())])
patients['avnrt'] = np.sort([np.int(x) for x in (X_all[X_all['Type']=='avnrt']['Patient'].unique())])
patients['avrt'] = np.sort([np.int(x) for x in (X_all[X_all['Type']=='avrt']['Patient'].unique())])
patients['ep'] = np.sort([np.int(x) for x in (X_all[X_all['Type']=='ep']['Patient'].unique())])

patient_types = ['af', 'at', 'avnrt', 'avrt', 'ep']

model_scores_list = []
for patient_type in patient_types:
    for patient in patients[patient_type]:
        patient_X = X_all[(X_all['Type']==patient_type) & (X_all['Patient']==str(patient))]
        if (patient_type + str(patient))=='avnrt11':
            pdb.set_trace()
            print('WTF')
            
        max_f1 = np.max(patient_X['Weighted PP'].values)
        max_delay = np.max(patient_X['Location of Maximum Energy'].values)
        
        patient_dict = {}
        patient_dict['Type'] = patient_type.upper()
        patient_dict['Patient'] = patient
        patient_dict['Max F1'] = max_f1*5/2
        patient_dict['Max Delay'] = max_delay
        
        model_scores_list.append(patient_dict)

> <ipython-input-158-ce6af585b78e>(16)<module>()->None
-> print('WTF')
(Pdb) patient
11
(Pdb) patient_X['Weighted PP']
476    0.000121
477    0.033489
478    0.068746
479    0.000847
480    0.127229
481    0.140228
482    0.000527
483    0.100201
484    0.145272
485    0.001400
486    0.348900
487    0.156133
488    0.000475
489    0.198870
490    0.206552
491    0.000913
492    1.087185
493    0.222461
494    0.000336
495    0.094194
496    0.152692
497    0.253333
498    1.712662
499    0.180066
500    0.001684
501    0.757491
502    0.180569
Name: Weighted PP, dtype: float64
(Pdb) patient_X[['Coupling Interval', 'Channel', 'Weighted PP']]
    Coupling Interval Channel  Weighted PP
476               400   CS1-2     0.000121
477               400   CS3-4     0.033489
478               400   CS5-6     0.068746
479               380   CS1-2     0.000847
480               380   CS3-4     0.127229
481               380   CS5-6     0.140228
482               360   CS1-2     0.000527
483   

In [151]:
model_scores = pd.DataFrame(model_scores_list)
# model_scores.to_csv(r'/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/model_analysis_results.csv')

In [16]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "t/p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()